In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image


In [4]:
data_dir = "training_data/"

#Resize data and set batches for processing
img_height, img_width = 150, 150
batch_size = 32

#spliting the data for training/validation
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    #Loading the training set
    data_dir,
    #The .2 reserves 20% of the dataset to be used for validiation, the rest for training
    validation_split=0.2,
    subset="training",
    #Setting a seed to be able to reproduces
    seed=1337,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    #Load the training set
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

#A bunch of blackbox code that has tensorflow run better
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 903 files belonging to 2 classes.
Using 723 files for training.
Found 903 files belonging to 2 classes.
Using 180 files for validation.


In [5]:
#Creating a sequential CNN model(go for image processing)
model = models.Sequential([
    
    #adjusting pixel value for better processing
    layers.experimental.preprocessing.Rescaling(1./255),
    
    #Convolution Layer, seperate the img into 3x3(pixels) filters to identify features
    #Using a relu(Rectified Linear Activation Function) to adjust for error and adjustments
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    
    #improve efficiency by cutting the output
    layers.MaxPooling2D((2, 2)),
    
    #Series of layers that take in output of past layers and run through another
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    #Turn output into array
    layers.Flatten(),
    
    #create layer of nodes
    layers.Dense(64, activation='relu'),
    
    #turn nodes into the classified groups
    layers.Dense(2)  # two classes: occupied and empty
])


In [6]:
"""
turning the network into a model, 'adam' algorithm for learing.
loss, measures accuracy, SpaceCategoryCrossentropy since to label classes
meteric is used for accuracy.

"""

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#Run the model with the training and validiation dataset, 10 times
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10


2023-09-05 13:27:08.240443: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [723]
	 [[{{node Placeholder/_0}}]]
2023-09-05 13:27:08.240628: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [723]
	 [[{{node Placeholder/_4}}]]


23/23 [==============================] - ETA: 0s - loss: 0.6055 - accuracy: 0.6750

2023-09-05 13:27:15.018728: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [180]
	 [[{{node Placeholder/_4}}]]
2023-09-05 13:27:15.019183: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [180]
	 [[{{node Placeholder/_0}}]]


23/23 [==============================] - 8s 320ms/step - loss: 0.6055 - accuracy: 0.6750 - val_loss: 0.5256 - val_accuracy: 0.7333
Epoch 2/10
23/23 [==============================] - 18s 816ms/step - loss: 0.4578 - accuracy: 0.6971 - val_loss: 0.4380 - val_accuracy: 0.8111
Epoch 3/10
23/23 [==============================] - 19s 857ms/step - loss: 0.3686 - accuracy: 0.8409 - val_loss: 0.4086 - val_accuracy: 0.8167
Epoch 4/10
23/23 [==============================] - 18s 783ms/step - loss: 0.2234 - accuracy: 0.9336 - val_loss: 0.4733 - val_accuracy: 0.8111
Epoch 5/10
23/23 [==============================] - 18s 797ms/step - loss: 0.0964 - accuracy: 0.9723 - val_loss: 0.4254 - val_accuracy: 0.8389
Epoch 6/10
23/23 [==============================] - 18s 785ms/step - loss: 0.0564 - accuracy: 0.9779 - val_loss: 0.5584 - val_accuracy: 0.8000
Epoch 7/10
23/23 [==============================] - 19s 823ms/step - loss: 0.1452 - accuracy: 0.9433 - val_loss: 0.6853 - val_accuracy: 0.7333
Epoch 8/10


In [7]:
val_loss, val_acc = model.evaluate(val_ds, verbose=2)
print("\nValidation accuracy:", val_acc)


6/6 - 1s - loss: 0.8692 - accuracy: 0.7944 - 599ms/epoch - 100ms/step

Validation accuracy: 0.7944444417953491


In [8]:
model.save('parking_model.h5')